In [2]:
import pandas as pd  # таблицы

dir = "D:\HSE\Pump_Dump\\"  # папка проекта

# фрейм ЦБ ММВБ
shares_pd = pd.read_csv(dir + "market\\moex_cb.csv", sep=",", encoding="utf-8")
shares_pd = shares_pd[(shares_pd["SUPERTYPE"] == "Акции")]
shares_pd.dropna(subset="INN", axis=0, inplace=True)
# удаляем строки без тикеров
shares_pd.dropna(subset="TRADE_CODE", axis=0, inplace=True)
# Тикеры к нижнему регистру
shares_pd["TRADE_CODE"] = shares_pd["TRADE_CODE"].str.lower()
shares_list = list(
    shares_pd[shares_pd["LIST_SECTION"] == "Третий уровень"]["TRADE_CODE"]
)

In [66]:
# корпус конкретного канала. Параметр - наименование канала
def channel_stat(channel):
    channel_path = dir + "TG\\" + channel + "_post.csv"
    # загрузка файла
    df = pd.read_csv(
        channel_path,
        encoding="utf-8",
        sep="|",
        keep_default_na=False,
    )
    len_all = len(df)
    # преобразуем поле с датой новости в тип ДАТА-ВРЕМЯ
    df["NEWS_DATE"] = pd.to_datetime(df["NEWS_DATE"], dayfirst=False, errors="ignore")
    # интервал дат
    # крайние N дней
    days_count = 180
    to_date = pd.to_datetime("today").date()
    from_date = to_date - pd.Timedelta(days=days_count)
    from_date = from_date.strftime("%Y-%m-%d")
    to_date = to_date.strftime("%Y-%m-%d")

    df = df[(df["NEWS_DATE"] < to_date) & (df["NEWS_DATE"] >= from_date)]
    len_date = len(df)
    # только единичные тикеры
    df = df[(df["TICKERS"].str.count(",") < 1) & (df["TICKERS"] != "")]
    len_tic = len(df)
    # только тикеры из 3-го эшелона
    df = df[df["TICKERS"].isin(shares_list)]
    len_tic_lev = len(df)
    # только тикеры, которые встречаются больше N раз
    df = df.groupby("TICKERS").filter(lambda x: len(x) > 3)
    len_tic_com = len(df)
    # print(channel, len_all, len_date, len_tic, len_tic_lev, len_tic_com)  # воронка
    data = [
        {
            "Канал": channel,
            "Всего записей": len_all,
            "В интервале дат": len_date,
            "С одним тикером": len_tic,
            "Из 3-го эшелона": len_tic_lev,
            "Больше N раз": len_tic_com,
        }
    ]
    i_df = pd.DataFrame.from_dict(data)
    return df, i_df

In [67]:
channel_path = dir + "TG\\" + "!channels" + ".csv"
channels_df = pd.read_csv(
    channel_path,
    encoding="utf-8",
    sep="|",
    keep_default_na=False,
)
big_df = pd.DataFrame()
funnel = pd.DataFrame()
for channel in channels_df["CHANNEL"].values:
    # обрабатываем канал и добавляем его в статистический фрейм
    # посты
    big_df = pd.concat([big_df, channel_stat(channel)[0]], ignore_index=False)
    # воронка
    funnel = pd.concat([funnel, channel_stat(channel)[1]], ignore_index=True)

In [71]:
big_df.sort_values(by=["TICKERS", "NEWS_DATE"], ascending=False)[
    ["CHANNEL", "TICKERS", "NEWS_DATE", "NEWS_TIME", "NEWS_TEXT"]
].to_html(dir + "Answers\\" + "3_Tickers_News.html")

In [72]:
big_df.groupby(["TICKERS", "CHANNEL", "NEWS_WEEK"]).count()[["POST"]].to_html(dir+"Answers\\"+"2_Tickers_Channels.html")

In [74]:
funnel.sort_values(by=["Из 3-го эшелона"], ascending=False).to_html(dir+"Answers\\"+"1_Tickers_Stat.html")